(본문서는 python 2기반의 jupyter에서 작성된 문서입니다)

#Chapter 5. Multiple View Geometry

Chapter 3, 4 와 계속 연결되는 내용임

#5.1 Epipolar Geometry

Epipolar Geometry에 대해서 이해 하기 위해서는 캠브리지 대학에서 출판한 *'Multiple View Geometry in Computer Vision'*책을 참조하는 것이 좋음(Computer Vision 분야에서는 바이블 취급을 받고 있는 교재임)

교재 제목으로 구글링을 하면, 샘플 chapter로 Epipolar Geometry에 관한 pdf파일을 받을 수 있음.

*Epipolar Geometry 용어 : 직역하면 '등극선 기하학'이라 불리지만, 대부분 '에피폴라 기하학'으로 부르고 있음. 해당 장에서의 의미는 3차원공간의 동일한 객체를 다른 시점에서 촬영한 두장이상의 사진간의 기하학적 관계에 대해 기하학적으로 설명을 하는 분야.

 카메라에 대한 정보(초점거리, 픽셀크기, 거리 등)없을 경우 3차원의 점 $\mathbf{X}$의 좌표가 애매하게 됨 이를 알기위해
 
 임의의 Homography $\mathbf{H}$(4 x 4), 카메라에서의 점좌표 $\mathbf{P}$를 가지고 수식으로 표현을 하면 다음과 같다.

\begin{equation*}
\lambda \mathbf{x}=P\mathbf{X}=PH^{-1}H\mathbf{X}=\hat{P}\hat{\mathbf{X}}
\end{equation*}

\begin{equation*}
P_{1}=K_{1}[I|0]
\end{equation*}

\begin{equation*}
P_{2}=K_{2}[R|\mathbf{t}]
\end{equation*}

 Chapter 4에서 처럼 $K_{1}$와 $K_{2}$은 카메라 캘리브레이션 매트릭스가 된다. 두번째 수식의  $R$은 두번째 카메라의 회전, $\mathbf{t}$는 평행이동을 의미한다.

 카메라 매트릭스에 의해서 3차원의 점인 $\mathbf{X}$는 각각 두 이미지에서 $\mathbf{x}_1$과 $\mathbf{x}_2$가 된다.


\begin{equation*}
\mathbf{x}_{2}^{T}\mathbf{F}\mathbf{x}_{1}=0
\end{equation*}


\begin{equation*}
\mathbf{F}=\mathbf{K}_{2}^{-T}\mathbf{S}_{t}\mathbf{R}\mathbf{K}_{1}^{-1}=0
\end{equation*}

 위의 수식에서 matrix $\mathbf{S}_{t}$는 skew symmetric matrix이다.

\begin{equation*}
\mathbf{S}_{t}
=
\begin{bmatrix}
0 & -t_{3} & t_{2} \\
t_{3} & 0 & -t_{1} \\
-t_{2} & t_{1} & 0
\end{bmatrix}
\end{equation*}

여기서 처음 등장한 $\mathbf{F}$는 *fundamental matrix*라고 부릅니다. 이 matrix에는 두 (카메라의) 이미지 matrices로 표현됩니다. fundamental matrix는 rank 2이고  $det(\mathbf{F})=0$을 성립합니다.



![epipolar02](data/notebook/epipolar02.jpg)

![epipolar](data\notebook\epipolar.jpg)

\begin{equation*}
\mathbf{x}_{2}^{T}\mathbf{F}\mathbf{x}_{1}=\mathbf{l}_{1}^{T}\mathbf{x}_{1}=0
\end{equation*}

 교재에서는 수식이 먼저 나온뒤에 설명이 있는데, 그림을 먼저 보면서 수식을 이해해야 한다. 여기서 line $\mathbf{l}$은 *epipolar line*이라 부른다. 앞의 fundamental matrix는 동일한 점$\mathbf{X}$에 대한 수식이기 때문에 위의 수식과 같은 관계가 성립된다.
 
  epipolar lines에서 만나는 점$\mathbf{e}$는 epipole이라 부른다.
  
 $\mathbf{F}\mathbf{e}_{1}=0$
 
 $\mathbf{e}_{2}^{T}\mathbf{F}=0$

![epipolar explain](data/notebook/epipolarExplain.jpg)

##A Sample Data Set

예제에서 사용하는 이미지와 데이터는 Oxford에서 제공하는 multi-view dataset중 Merton1을 사용한다.

다운로드 주소: http://www.robots.ox.ac.uk/~vgg/data/data-mview.html

(아래의 코드에서 import 하는 Camera 모듈은 앞의 4장에서 선언한 camera class인것으로 생각된다)

In [10]:
import Camera

# load some images
im1 = array(Image.open('data/Merton1/images/001.jpg'))
im2 = array(Image.open('data/Merton1/images/002.jpg'))

# load 2D points for each view to a list
points2D = [loadtxt('data/Merton1/2D/00'+str(i+1)+'.corners').T for i in range(3)]

# load 3D points
points3D = loadtxt('data/Merton1/3D/p3d').T

# load correspondences
corr = genfromtxt('data/Merton1/2D/nview-corrers',dtype='int',missing='*')

# load cameras to a list of Camera objects
P = [camera.Camera(loadtxt('data/Merton1/2D/00'+str(i+1)+'.P')) for i in range(3)]

NameError: name 'array' is not defined

##Plotting 3D Data with Matplotlib

본 실습에서 사용한 점은 이미 구해진 점 데이터를 사용한다. (직접 점을 구하여 맞는지 여부를 확인하기 위해서는 많은 시간이 걸린다)

![morton1](data/notebook/figure02.jpg)

##Computing F-The Eight Point Algorithm

8 Point Algorithm의 자세한 내용은 앞서 언급된 *'Multiple View Geometry in Computer Vision'*교재에 자세히 나와 있다.

![Merton1 data plots](data/notebook/figure03.jpg)

##The Epipole and Epipolar Lines

앞에서 언급한 Epipolar Lines을 찾는 방법에 대해서 언급하는 내용이다.

![epipolar line sample](data/notebook/figure04.jpg)

#5.2 Computing with Cameras and 3D Structure

 앞에 까지는 fundamental matrix와 epipolar line를 어떻게 계산하고 어떠한 관계가 있는지 보았다. 이번 절에서는 3차원 구죠를 계산하는 방법에 대해서 설명한다.

##Triangulation

![triangulation](data/notebook/figure05.jpg)

##Computing the Camera Matrix from 3D Points

![estimated camera matrix](data/notebook/figure06.jpg)

##Computing the Camera Matrix from a Fundamental Matrix

###The uncalibrated case - projective reconstruction

###The Calibrated case - metric reconstruction

#5.3 Multiple View Reconstruction

the camera has been calibrated, the step
1. Detect feature points and match them between the two images.
2. Compute the fundamental matrix from the matches.
3. Compute the camera matrices from the fundamental matrix.
4. Triangulate the 3D points.

##Robust Fundamental Matrix Estimation

Fundamental Matrix song

https://www.youtube.com/watch?v=p16frKJLVi0


![fundamental matrix](data/notebook/figure07.jpg)

##3D Reconstruction Example

##Extensions and More Than Two Views

###More views

![sequnce](data/notebook/figure08.jpg)

###Bundle adjustment

###Self-calibration

#5.4 Stereo Images

![stereo image](data/notebook/figure09.jpg)

##Computing Disparity Maps

![example](data/notebook/figure10.jpg)

![example2](data/notebook/figure11.jpg)